Noam Shayer 315206904  
Tal Elrom 207517673  
Github https://github.com/TalElrom/Collecting-Data-Project

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [2]:
url = "https://www.ad.co.il/nadlanrent?sp275=17413&sp276=17414&sp277=17874,17783,18685"


In [3]:
def get_properties(url):
    properties_list = list()
    response = requests.get(url)
    if not response.status_code == 200:
        print ("not 200")
        return None
    try:
        results_page = BeautifulSoup(response.content, 'html.parser')
        properties = results_page.find_all('div',{'class':'card-body p-md-3'})
        for property1 in properties:
            property_link = "https://www.ad.co.il" + property1.find('a').get('href')
            neighborhood = property1.find('a').get_text()
            if "תל אביב" not in neighborhood:
                continue  
            properties_list.append((property_link)) 
        return properties_list
    except:
        return None

In [4]:
properties_list = get_properties(url)
#properties_list

['https://www.ad.co.il/ad/16191048',
 'https://www.ad.co.il/ad/16191132',
 'https://www.ad.co.il/ad/16191138',
 'https://www.ad.co.il/ad/16060577',
 'https://www.ad.co.il/ad/16046349',
 'https://www.ad.co.il/ad/16031648',
 'https://www.ad.co.il/ad/15577632',
 'https://www.ad.co.il/ad/13899596',
 'https://www.ad.co.il/ad/16191462',
 'https://www.ad.co.il/ad/16190756',
 'https://www.ad.co.il/ad/13435926',
 'https://www.ad.co.il/ad/13411920',
 'https://www.ad.co.il/ad/13200500',
 'https://www.ad.co.il/ad/16047419',
 'https://www.ad.co.il/ad/16007115',
 'https://www.ad.co.il/ad/15993372',
 'https://www.ad.co.il/ad/15993373',
 'https://www.ad.co.il/ad/15965203',
 'https://www.ad.co.il/ad/15865081',
 'https://www.ad.co.il/ad/15550030',
 'https://www.ad.co.il/ad/15512003',
 'https://www.ad.co.il/ad/15471720',
 'https://www.ad.co.il/ad/15406302',
 'https://www.ad.co.il/ad/15394753',
 'https://www.ad.co.il/ad/15048109',
 'https://www.ad.co.il/ad/14776471',
 'https://www.ad.co.il/ad/14466322',
 

In [5]:
def get_distance_to_center(property_address):
    url = "https://routes.googleapis.com/directions/v2:computeRoutes"

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': 'your_key',  # Replace with your real key
        'X-Goog-FieldMask': 'routes.distanceMeters'
    }

    body = {
        "origin": {
            "address": property_address
        },
        "destination": {
            "address": "Dizengoff Square, Tel Aviv"
        },
        "travelMode": "DRIVE",
        "routingPreference": "TRAFFIC_AWARE"
    }

    response = requests.post(url, headers=headers, data=json.dumps(body))

    if response.status_code == 200:
        try:
            data = response.json()
            distance_meters = data['routes'][0]['distanceMeters']
            return distance_meters
        except Exception as e:
            print("Could not extract distance:", e)
            print(property_address)
            return None
    else:
        print("Error with API call:", response.status_code)
        return None


In [88]:
def get_property_details(property_link):
    import requests
    from bs4 import BeautifulSoup

    details = dict()

    try:
        response = requests.get(property_link)
        if response.status_code != 200:
            return details
        
        soup = BeautifulSoup(response.content, 'html.parser')

        table = soup.find('table', class_='table table-sm mb-4')
        floor = None
        total_floors = None
        address = None
        #print(table)

        if table:
            rows = table.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                if len(cells) == 2:
                    label = cells[0].get_text(strip=True)
                    value = cells[1].get_text(strip=True)

                    if label == 'פרטי הנכס':
                        details['property_type'] = str(value)
                    elif label == 'שכונה':
                        details['neighborhood'] = str(value)
                    elif label == 'כתובת':
                        details['address'] = str(value)
                        address = value
                    elif label == 'חדרים':
                        details['room_num'] = float(value)
                    elif label == 'קומה':
                        value_parts = value.split("מתוך")
                        if len(value_parts) == 2:
                            floor = value_parts[0].strip()
                            total_floors = value_parts[1].strip()
                            if floor == 'קרקע':
                                details['floor'] = 0
                            else:
                                try:
                                    details['floor'] = int(floor)
                                except:
                                    details['floor'] = None

                            try:
                                details['total_floors'] = int(total_floors)
                            except:
                                details['total_floors'] = 0
                    elif label == 'שטח בנוי':
                        details['area'] = int(value)
                    elif label == 'שטח גינה':
                        details['garden_area'] = int(value)
                    elif label == 'תאריך כניסה':
                        if value == 'מיידית':
                            details['days_to_enter'] = 0
                        else: 
                            details['days_to_enter'] = value
                    elif label == 'תשלומים בשנה':
                        details['num_of_payments'] = int(value)
                    elif label == 'ארנונה בחודש':
                        details['monthly_arnona'] = int(value)
                    elif label == 'ועד בית בחודש':
                        details['building_tax'] = int(value)
                        
            if not address and 'neighborhood' in details:
                address = details['neighborhood']
                details['address'] = address

            desc = soup.find('p', class_='text-word-break')
            if desc:
                details['description'] = str(desc.get_text(strip=True))
            
            features = soup.find_all('div', class_='card-icon')
            for feature in features:
                label_span = feature.find('span')
                if not label_span:
                    continue
                feature_name = label_span.get_text(strip=True)
                is_disabled = 'disabled' in feature.get('class', [])
                if feature_name == 'חניה':
                    details['has_parking'] = 0 if is_disabled else 1
                elif feature_name == 'מחסן':
                    details['has_stotsge'] = 0 if is_disabled else 1
                elif feature_name == 'מעלית':
                    details['elevator'] = 0 if is_disabled else 1
                elif feature_name == 'מזגן':
                    details['ac'] = 0 if is_disabled else 1                    
                elif feature_name == 'נגישות':
                    details['handicap'] = 0 if is_disabled else 1
                elif feature_name == 'סורגים':
                    details['has_bars'] = 0 if is_disabled else 1
                elif feature_name == 'ממ"ד':
                    details['has_safe_room'] = 0 if is_disabled else 1                    
                elif feature_name == 'מרפסת':
                    details['has_balcon'] = 0 if is_disabled else 1
                elif feature_name == 'מרוהטת':
                    details['is_furnished'] = 0 if is_disabled else 1                    
                elif feature_name == 'משופצת':
                    details['is_renovated'] = 0 if is_disabled else 1
                    
            price_container = soup.find('div', class_='d-flex justify-content-between')
            if price_container:
                h2_tags = price_container.find_all('h2', class_='card-title')
                if len(h2_tags) >= 2:
                    price_text = h2_tags[1].get_text(strip=True).replace("₪", "").replace(",", "")
                    details['price'] = float(price_text)
                    
            photo_slides = soup.find_all('div', class_='swiper-slide')
            photo_count = 0
            for slide in photo_slides:
                a_tag = slide.find('a', href=True)
                if a_tag and 'NadlanRentImages' in a_tag['href']:
                    photo_count += 1
            details['num_of_images'] = photo_count
            full_address = f"{address}, תל אביב"
            distance = get_distance_to_center(full_address)
            if distance:
                details['distance_from_center'] = float(distance)
            
                    
                    

        return details
    except Exception as e:
        print("Error:", e)
        return details


In [89]:
def get_all_property_details(properties_list):
    all_details = []
    for property_link in properties_list:
        try:
            property_info = get_property_details(property_link)
            if property_info:
                all_details.append(property_info)
        except Exception as e:
            print(f"Error processing {property_link}: {e}")
    return all_details


In [90]:
all_properties_data = get_all_property_details(properties_list)


In [91]:
#all_properties_data

[{'property_type': 'גג/ פנטהאוז',
  'neighborhood': 'כרם התימנים',
  'address': 'עזרא הסופר 15',
  'room_num': 3.0,
  'floor': 6,
  'total_floors': 6,
  'area': 100,
  'days_to_enter': 0,
  'num_of_payments': 12,
  'description': 'ללא תיווך ישירות מהבעלים ברחוב עזרא הסופר 15 דירת פנטהאוז כיפית בבניין בוטיק. דירה יחידה בקומה, 3 חדרים, משקיפה לים 360 של נוף פתוח. 2 מרפסות שמש ומרפסת שירות. מטבח יפה ונוח. ויש גם חניה בבניין. הדירה נמצאת ברחוב שקט ובמרחק מספר צעדים מחוף הים, ליד שוק הכרמל התוסס, כרם התימנים, סמטאות יפו העתיקה, שוק הפשפשים ונווה צדק עם כל המסעדות ובתי הקפה המדליקים. יש קודן בכניסה לבניין',
  'is_furnished': 0,
  'ac': 1,
  'has_parking': 0,
  'has_safe_room': 0,
  'has_balcon': 1,
  'handicap': 0,
  'has_bars': 0,
  'elevator': 1,
  'has_stotsge': 0,
  'is_renovated': 0,
  'price': 12000.0,
  'num_of_images': 10,
  'distance_from_center': 1752.0},
 {'property_type': 'דירת גן',
  'neighborhood': 'כרם התימנים',
  'address': 'האר"י 28',
  'room_num': 4.0,
  'floor': 1,
  'tota

In [92]:
#len(all_properties_data)

39

In [93]:
column_order = [
    'property_type', 'neighborhood', 'address', 'room_num', 'floor',
     'area', 'garden_area', 'days_to_enter', 'num_of_payments',
    'monthly_arnona', 'building_tax','total_floors', 'description', 'has_parking',
    'has_stotsge', 'elevator', 'ac', 'handicap', 'has_bars',
    'has_safe_room', 'has_balcon', 'is_furnished', 'is_renovated',
    'price', 'num_of_images', 'distance_from_center'
]

df = pd.DataFrame(all_properties_data)[column_order]

In [95]:
df['property_type'] = df['property_type'].astype('string')
df['neighborhood'] = df['neighborhood'].astype('string')
df['address'] = df['address'].astype('string')
df['description'] = df['description'].astype('string')

In [96]:
df['garden_area'] = df['garden_area'].fillna(0).astype(int)
df['days_to_enter'] = df['days_to_enter'].astype('Int64')  
df['monthly_arnona'] = df['monthly_arnona'].astype('Int64')
df['building_tax'] = df['building_tax'].astype('Int64')

In [87]:
df.to_csv("ProjectPart1.csv", index=False, encoding='utf-8-sig')

In [104]:
df[['monthly_arnona']]

,monthly_arnona
0,<NA>
1,400
2,400
3,500
4,450
5,200
6,225
7,800
8,<NA>
9,450
